In [1]:
# Thanks to user394430 on StackOverflow for this!
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)


from ipywidgets import *
from sympy import *
init_printing()
from sympy.abc import a, b, c, x, y, z
from random import *
import functools

LoL = [a, b, c, x, y, z]

degree = randint(2,4) # Randomly assign the degree of the simplified polynomial

numvars = randint(1,2) # Randomly assign the number of variables to work with

# Randomly choose between one or max elements of a list
def choose (ListOfLetters, max):
        picks = []
        while len(picks)<1 or len(picks)>max:
            picks = sample(range(0, len(ListOfLetters)), randint(1, len(ListOfLetters)))
        return [ListOfLetters[p] for p in picks]
    
# Convert input expression to Latex code
def tex(expression):
    return latex(eval(str(expression)))

# create a list of variables
varlist = choose(LoL, numvars) # Choose variables
varlist = [prod(varlist)] if randint(1,2) ==1 else varlist
while len(varlist)<2:
    varlist = varlist + [1]
    
# Create a list of coefficients    
coeffs = [x for x in range(-3,3)] if randint(1,3) in {1, 2} else [Rational(x, y) for x in range(-3, 3) for y in range(-3, 3) if y != 0] 
coefflist1 = [sample(coeffs, 2) for x in range(0,degree)]
coefflist2 = [sample(coeffs, 2) for x in range(0,degree)]

# Multiply the variables by the coeffients
elements1 = [[(x*y) for (x, y) in zip(varlist, coefflist1[k])] for k in range(len(coefflist1))]
elements2 = [[(x*y) for (x, y) in zip(varlist, coefflist2[k])] for k in range(len(coefflist2))]

# Create linear factors 
factors1 = [x+y for (x, y) in elements1]
factors2 = [x+y for (x, y) in elements2]

# Expand the linear factors
term1 = expand(prod(factors1))
term2 = expand(prod(factors2))
solution = Add(term1, term2)
solution_exploded = (sympify(("+").join([str(term1)]+[str(term2)]), evaluate = False))

# Create a dictionary of variables and coefficients
explode_term1 = term1.as_coefficients_dict()
explode_term2 = term2.as_coefficients_dict()
gathered  = sum([Mul(Add(v1, v2, evaluate = False), 
                     k1, evaluate = False) for (k1, v1) in explode_term1.items() 
                     for (k2,v2) in explode_term2.items() if simplify(k1-k2) == 0])

# Catch terms in each dictionary that did not have the same keys
missing1 =set(explode_term1.keys()).difference(set(explode_term2))
filler1 =[explode_term1[m]*m for m in missing1]
missing2 =set(explode_term2.keys()).difference(set(explode_term1))
filler2 = [explode_term2[m]*m for m in missing2]
full = sum([gathered]+filler1+filler2)

# Produce the problem for display
problem = tex(term1) + " + " + tex(term2) if LC(term2)>0 else tex(term1) + tex(term2)
split_problem = problem.split(" - ")
catcher = 1
while catcher !=-1:
    shuffled = (" - ").join(sample(split_problem, len(split_problem)))
    catcher = shuffled.find("- -")
    
# Create the problem 
instructions =widgets.HTML(
    value="Completely simplify:"
)
question = widgets.HTMLMath(
            value=r"\(" + shuffled + "\)"
)
answer = widgets.Text(
       value='0',
       placeholder="",
       description='Answer', )
button = widgets.Button(description='Check Answer')
out = widgets.Output()
def on_button_clicked(_):
            with out:
                out.clear_output()
                print("Correct!") if sympify(answer.value) - solution == 0 else print("Incorrect")
                display(widgets.HTML(
                    value="Given:"
                    ))
                display(widgets.HTMLMath(
                  value = r"\(" + shuffled +"\)"
                    ))
                display(widgets.HTML(
                    value="Gather like terms (This line can be skipped):"
                    ))
                display(widgets.HTMLMath(
                  value = r"\(" + latex(solution_exploded) + "\)"
                    ))
                display(widgets.HTML(
                    value="Sum the coefficients of the like terms (This line can be skipped):"
                    ))
                display(widgets.HTMLMath(
                  value = r"\(" + latex(full) + "\)"
                    ))
                display(widgets.HTML(
                    value="Resulting in (This line cannot be skipped):"
                    ))
                display(widgets.HTMLMath(
                  value = r"\(" + latex(full.doit()) + "\)"
                    ))
button.on_click(on_button_clicked)
widgets.VBox([instructions, question, answer, button, out])

Toggle code